In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [0]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [77]:
page = requests.get('https://vc.ru/trade/63330')
soup = BeautifulSoup(page.text, 'html5lib')
info = []
a = soup.find("span", class_="vote__value__v vote__value__v--real").text
if a[0] == '–':
  b = int(a[1:])
  info.append(-b)
else:
    info.append(int(a))
print(info)

[-1]


In [0]:

def document(id):
    page = requests.get('https://vc.ru/trade/' + str(id) )
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("h1", class_="content-header__title"):
        #titledone
        info.append(soup.find("h1", class_="content-header__title").text.split())
        #text не знаю как решить, почему-то выдает заглавие и раздел, 
        #хотя у них другой класс
        text = soup.find("div", class_="layout--a").text
        info.append(cleanhtml(text).split())
        #yeardone
        info.append(soup.find("time", class_= "time").text.split()[2])
        #sectiondone
        info.append(soup.find("div", class_=
                              "content-header-author__name").text.split())
        #votesdone
        info.append(soup.find("span", class_=
                              "vote__value__v vote__value__v--real").text)
        #viewsdone
        info.append(soup.find("span", class_="views__value").text)
        #bookmarksdone
        a = soup.find("span", class_="vote__value__v vote__value__v--real").text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #commentsdone
        comment1 = soup.find_all("div", class_="comments__item__text")
        comments = []
        for i in range(len(comment1)):
            comments.append(comment1[i].text)
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info
    

In [0]:
a = list(map(document, np.arange(#начало, #конец)))


In [0]:
df = pd.DataFrame(a, columns=['title', 'text', 'year', 'section', 'votes', 'views', 'bookmarks', 'comments'])
df.dropna(inplace=True)
print(df)